In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchquantum as tq

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import cv2

from skimage import io
from PIL import Image


In [2]:
# define controlled hadamard gate
sq2 = 1/np.sqrt(2)
def controlled_H(qdev, target,control):
      qdev.apply(tq.QubitUnitary(
      has_params=True,init_params=([[1,0,0,0],[0,sq2,0,sq2],[0,0,1,0],[0,sq2,0,-sq2]]),wires=[target,control]))
 

Object `view` not found.


In [52]:
# x is the input 
x = torch.tensor([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
x = torch.rand(100,256,256)

print(x.shape)
#x = x.view(4,16) # to reshape the tensor
print(x[0])

torch.Size([100, 256, 256])
tensor([[0.3621, 0.1498, 0.5706,  ..., 0.4156, 0.5716, 0.8559],
        [0.3958, 0.8412, 0.7895,  ..., 0.1166, 0.7951, 0.3923],
        [0.9597, 0.2227, 0.6577,  ..., 0.3778, 0.2328, 0.2865],
        ...,
        [0.6649, 0.6406, 0.9665,  ..., 0.0784, 0.1857, 0.0200],
        [0.1110, 0.8320, 0.9081,  ..., 0.8613, 0.7684, 0.9758],
        [0.3633, 0.7559, 0.1300,  ..., 0.3864, 0.7073, 0.4571]])


In [53]:
print(x[2])

tensor([[0.8947, 0.7980, 0.2596,  ..., 0.5840, 0.6783, 0.1627],
        [0.4275, 0.7236, 0.1614,  ..., 0.1128, 0.8175, 0.6031],
        [0.7143, 0.1140, 0.6310,  ..., 0.3187, 0.3684, 0.1996],
        ...,
        [0.3339, 0.9225, 0.6631,  ..., 0.8142, 0.4090, 0.5994],
        [0.0814, 0.0967, 0.2155,  ..., 0.6682, 0.4648, 0.0070],
        [0.0378, 0.9988, 0.6430,  ..., 0.6185, 0.0587, 0.9132]])


In [54]:
#print(x[0]) # first image
#print(x[0,0]) # first row of the first image
print(x[0,0,0]) # first pixel of the first row of the first image
#print(x[0,:,0]) # first column of the first image
print(x[:,0,0]) # list of pixels at (0,0) of all images
x[:,0,0+1] # list of pixels at (0,1) of all images
x[:,0+1,0] # list of pixels at (1,0) of all images
x[:,0+1,0+1] # list of pixels at (1,1) of all images
torch.cat((x[:, 0, 0], x[:, 0, 0+1], x[:, 0+1, 0], x[:, 0+1, 0+1])) # list of pixels at (0,0), (0,1), (1,0), (1,1) of all images

tensor(0.3621)
tensor([0.3621, 0.0995, 0.8947, 0.6125, 0.5663, 0.2897, 0.0862, 0.2509, 0.4025,
        0.9555, 0.8904, 0.7887, 0.4842, 0.6590, 0.9295, 0.7216, 0.2839, 0.2622,
        0.5806, 0.3185, 0.7709, 0.1310, 0.1731, 0.1140, 0.8124, 0.4603, 0.9993,
        0.8800, 0.4340, 0.8631, 0.0423, 0.8512, 0.3358, 0.5474, 0.7675, 0.3433,
        0.6147, 0.7819, 0.7963, 0.8118, 0.3661, 0.4824, 0.4168, 0.9186, 0.1280,
        0.8740, 0.9863, 0.0318, 0.3563, 0.2173, 0.6616, 0.5147, 0.8302, 0.9485,
        0.5114, 0.9541, 0.0516, 0.5001, 0.8533, 0.7540, 0.0649, 0.6133, 0.0306,
        0.2059, 0.0273, 0.9221, 0.0402, 0.4775, 0.1499, 0.0287, 0.3849, 0.0799,
        0.0254, 0.7256, 0.8309, 0.5344, 0.6416, 0.4504, 0.6615, 0.1124, 0.4128,
        0.4922, 0.3278, 0.2425, 0.8162, 0.5853, 0.1765, 0.2885, 0.1550, 0.1294,
        0.9369, 0.5583, 0.6550, 0.5714, 0.8849, 0.5793, 0.2621, 0.6059, 0.5127,
        0.5980])


tensor([0.8412, 0.6771, 0.7236, 0.1376, 0.2603, 0.1329, 0.6727, 0.7956, 0.1888,
        0.7975, 0.1610, 0.4217, 0.4693, 0.4494, 0.4325, 0.0736, 0.3461, 0.3917,
        0.3693, 0.2489, 0.4091, 0.2408, 0.6728, 0.0136, 0.8628, 0.6364, 0.3646,
        0.9044, 0.8733, 0.3287, 0.5531, 0.8773, 0.7473, 0.3041, 0.3030, 0.6875,
        0.7789, 0.1700, 0.0968, 0.1315, 0.5975, 0.4985, 0.9761, 0.8742, 0.5603,
        0.1389, 0.5817, 0.2464, 0.4763, 0.8450, 0.7419, 0.5835, 0.0248, 0.7725,
        0.0601, 0.7170, 0.4996, 0.1446, 0.6374, 0.6223, 0.1454, 0.1208, 0.3583,
        0.6223, 0.4601, 0.8802, 0.1035, 0.2576, 0.1737, 0.2086, 0.1683, 0.2175,
        0.0831, 0.4120, 0.8211, 0.7229, 0.0578, 0.3546, 0.3049, 0.3379, 0.8907,
        0.7936, 0.6809, 0.8609, 0.4184, 0.7515, 0.3582, 0.7728, 0.7218, 0.4607,
        0.3423, 0.6880, 0.1857, 0.4826, 0.5414, 0.7040, 0.9894, 0.6198, 0.3273,
        0.4306])

In [1]:
x1 = torch.cat((x[:, 0, 0], x[:, 0, 0+1], x[:, 0+1, 0], x[:, 0+1, 0+1])).view(4, 100) # first matrix/list is the list of all pixels at (0,0). second matrix/list is the list of all pixels at (0,1). third matrix/list is the list of all pixels at (1,0). fourth matrix/list is the list of all pixels at (1,1). 


NameError: name 'torch' is not defined

In [2]:
x1 = torch.transpose(torch.cat((x[:, 0, 0], x[:, 0, 0+1], x[:, 0+1, 0], x[:, 0+1, 0+1])).view(4, 100),0,1) # first row contains (0,0)(0,1)(1,0)(1,1) pixels


NameError: name 'torch' is not defined

In [3]:
bsz = x.shape[0] # batch size
size = 256 # height and width of an image
x = x.view(bsz, size, size) # reshape the data 

data_list = []

for c in range(0, size, 2):
    for r in range(0, size, 2):
        data = torch.transpose(torch.cat((x[:, c, r], x[:, c, r+1], x[:, c+1, r], x[:, c+1, r+1])).view(4, bsz), 0, 1)
        

        data_list.append(data.view(bsz, 4))
#
# print(data_list) # 100 images, 4 pixels, (2 x 2 encoded in one single list)

NameError: name 'x' is not defined

In [ ]:
# torch.cat takes following arguments: (tuple of tensors, dim=0, out=None)
# dim=0 means that the tensors are concatenated along the first dimension
# torch.transpose takes following arguments: (input, dim0, dim1, out=None)
# dim0 and dim1 are the dimensions to be swapped

In [76]:
result = torch.cat(data_list, dim=1).float() # concatenate all the tensors in the list along the first dimension 0th dim remains same which is the batch size.
print(result.shape)
print(result[0].shape) # first row is the first image concatanated in a single list with all the pixels i.e. 256 x 256 image flattened into 65536  x 1 

torch.Size([100, 65536])
torch.Size([65536])


In [3]:
class QuanvolutionFilter(tq.QuantumModule):
  # the __init__ method initializes the quantum device, the general encoder,
  # a random quantum layer, and a measurement operator.
    def __init__(self):
        super().__init__()
        self.n_wires = 4  # two ancillas
        self.q_device = tq.QuantumDevice(n_wires=self.n_wires)
        # encoding the input data
        self.encoder = tq.GeneralEncoder(
        [   {'input_idx': [0], 'func': 'ry', 'wires': [0]},
            {'input_idx': [1], 'func': 'ry', 'wires': [1]},
            {'input_idx': [2], 'func': 'ry', 'wires': [2]},
            {'input_idx': [3], 'func': 'ry', 'wires': [3]},])
        


        # random circuit layer 
        self.q_layer = tq.RandomLayer(n_ops=8, wires=list(range(self.n_wires)))
        self.measure = tq.MeasureAll(tq.PauliZ)
        #self.expval = tq.expval()
    
# x has the dimension of (batch_size, 28, 28) representing a batch of greyscale images
#The method first reshapes the input data into a 2D array of shape 
#(batch_size, 784) by concatenating adjacent 2x2 blocks of pixels.
# data is the new reshaped tensor 
    def forward(self, x, use_qiskit=False):
        bsz = x.shape[0] # batch size
        size = 256 # height and width of an image
        x = x.view(bsz, size, size) # reshape the data 

        data_list = []

        for c in range(0, size, 2):
            for r in range(0, size, 2):
                data = torch.transpose(torch.cat((x[:, c, r], x[:, c, r+1], x[:, c+1, r], x[:, c+1, r+1])).view(4, bsz), 0, 1)
                if use_qiskit:
                    data = self.qiskit_processor.process_parameterized(
                        self.q_device, self.encoder, self.q_layer, self.measure, data)
                else:
                    self.encoder(self.q_device, data)
                    
                    #haar wavelet
                    # level 1 
                    self.q_device.h(wires = 3) 
                    self.q_device.swap([3,2])
                    self.q_device.swap([2,1])
                    self.q_device.swap([1,0])

                    # level 2 
                    controlled_H(self.q_device, target=2, control= 3)
                    self.q_device.cswap([3,2,1])
                    self.q_device.cswap([3,1,0])

                    # level 3
                    
                    #self.q_device.ccx([2,3,4])
                    #controlled_H(self.q_device, target=1, control= 4)
                    #self.q_device.ccx([2,3,4])
                    #perm
                    #self.q_device.ccx([2,3,4])
                    #self.q_device.cswap([4,1,0])
                    #self.q_device.ccx([2,3,4])

                    #level 4
                    #self.q_device.ccx([2,3,4])
                    #self.q_device.ccx([1,4,5])
                    #controlled_H(self.q_device, target=0, control= 5)
                    #self.q_device.ccx([1,4,5])
                    #self.q_device.ccx([2,3,4])



                    self.q_layer(self.q_device)
                    data = self.measure(self.q_device)
                    
                    #for i in range(4):
                    #    measure_result = []
                    #    measure_result.append(tq.expval(self.q_device,wires=i, observables= tq.PauliZ(wires=i)))

                    #    data = torch.tensor([measure_result])

                    
                data_list.append(data.view(bsz, 4)) # only keep the first 4 qubits
        
        result = torch.cat(data_list, dim=1).float()
        
        return result


In [3]:

class HybridModel(torch.nn.Module): 
    def __init__(self): 
        super().__init__() 
        self.qf = QuanvolutionFilter()
        self.linear = torch.nn.Linear(4*128*128, 2) 
    def forward(self, x, use_qiskit=False):
        with torch.no_grad():
          x = self.qf(x, use_qiskit)
        x = self.linear(x)
        return F.softmax(x, -1) 
    # F.log_softmax is the log of the softmax function, which is a common choice for the output of a classification model.
    
class HybridModel_without_qf(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(256*256, 2)
    
    def forward(self, x, use_qiskit=False):
        x = x.view(-1, 256*256)
        x = self.linear(x)
        return F.log_softmax(x, -1)

In [10]:
# importing the dataset 
class Oral_Can_Data(Dataset):
    '''Oral cancer dataset'''
    def __init__(self, csv_file, root_dir,transform = None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations) # returns the number of samples in the dataset
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        image = np.array(image)
        image = torch.tensor([image])
        can_type = self.annotations.iloc[index, 1] # labels:  1 for non-cancerous, 2 for cancerous
        can_type = torch.tensor([can_type])
        can_type = can_type

    
        sample = {'image': image, 'can_type': can_type}
        
        for index in range(len(self.annotations)):
            sample

        if self.transform:
            image = self.transform(image)
        
        return (sample) # returns the image and the label


In [11]:
# loading the dataset
dataset = Oral_Can_Data(csv_file= '/home/iisers/Documents/oral_cancer_project/labels .csv',root_dir='/home/iisers/Documents/oral_cancer_project/Combined_data_resized')

train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_set, val_set,  test_set = torch.utils.data.random_split(dataset, [train_size,val_size,test_size])

dataflow = dict({'train' : train_set, 'valid': val_set , 'test' : test_set})

In [12]:
for i in dataflow:
    print(i, len(dataflow[i]))

train 368
valid 79
test 80


In [13]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = HybridModel().to(device)
model_without_qf = HybridModel_without_qf().to(device)
n_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)
criterion = torch.nn.CrossEntropyLoss()

In [14]:
accu_list1 = []
loss_list1 = []
accu_list2 = []
loss_list2 = []

def train(dataflow, model, device, optimizer):
    for feed_dict in dataflow['train']:
        inputs = feed_dict['image'].to(device)
        targets = feed_dict['can_type'].to(device)
        #print(targets)
        #print(targets.shape)
        

        outputs = model(inputs)
        #print(outputs.shape)
        #print(outputs)

        #print(inputs)
        #loss = F.nll_loss(outputs, targets)
        loss = F.binary_cross_entropy(outputs[0][0].to(torch.float32), targets[0].to(torch.float32))
        #loss = criterion(outputs[0], targets.view(-1, 1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"loss: {loss.item()}", end='\r')


def valid_test(dataflow, split, model, device, qiskit=False):
    target_all = []
    output_all = []
    with torch.no_grad():
        for feed_dict in dataflow[split]:
            inputs = feed_dict['image'].to(device)
            targets = feed_dict['can_type'].to(device)

            outputs = model(inputs, use_qiskit=qiskit)

            target_all.append(targets[0].to(torch.float32))
            output_all.append(outputs[0][0].to(torch.float32))
        target_all = torch.cat(target_all, dim=0)
        output_all = torch.cat(output_all, dim=0)

    _, indices = output_all.topk(1, dim=1)
    masks = indices.eq(target_all.view(-1, 1).expand_as(indices))
    size = target_all.shape[0]
    corrects = masks.sum().item()
    accuracy = corrects / size
    loss = F.binary_cross_entropy(output_all, target_all).item()

    print(f"{split} set accuracy: {accuracy}")
    print(f"{split} set loss: {loss}")

    return accuracy, loss

for epoch in range(1, n_epochs + 1):
    # train
    print(f"Epoch {epoch}:")
    train(dataflow, model, device, optimizer)
    print(optimizer.param_groups[0]['lr'])

    # valid
    accu, loss = valid_test(dataflow, 'test', model, device, )
    accu_list1.append(accu)
    loss_list1.append(loss)
    scheduler.step()


Epoch 1:
